In [1]:
import matplotlib.pyplot as plt
%matplotlib inline  
%load_ext autoreload
%autoreload 2
import numpy as np
import scipy as sp
import scipy.linalg
import time
import random
def print_np(x):
    print ("Type is %s" % (type(x)))
    print ("Shape is %s" % (x.shape,))
#     print ("Values are: \n%s" % (x))

In [2]:
import sys
sys.path.append('../')
sys.path.append('../model')
# import model
import UnicycleModel
import cost
from iLQR import iLQR
from scipy.integrate import solve_ivp

In [3]:
x_t = np.zeros(3)
x_t[0] = 0.0
x_t[1] = 2.0
x_t[2] = 0.0
ix = 3
iu = 2
N = 500
delT = 0.1
tf = delT * N
myModel = UnicycleModel.unicycle('Hello',ix,iu,linearization="numeric_central")
myCost = cost.unicycle('Hello',x_t,N)

In [4]:
maxIter= 100

x0 = np.zeros(3)
x0[0] = -1.0 # -2.0
x0[1] = 0.0 # -0.5
x0[2] = np.pi/2

u0 = np.random.rand(N,iu)
i1 = iLQR('unicycle',delT,N,tf,maxIter,myModel,myCost,discretization="ACL")
x, u, Quu_save, Quu_inv_save, L, l = i1.update(x0,u0)

iteration   cost        reduction   expected    gradient    log10(lambda)
0           464.901     241         166         0.307       0.0         
1           224.275     79          51.2        0.206       -0.2        
2           145.267     24.6        19.2        0.146       -0.6        
3           120.643     17.4        17.2        0.178       -1.2        
4           103.223     13          12          0.151       -2.0        
5           90.231      13.6        12.3        0.157       -3.1        
6           76.5858     17.5        15          0.136       -4.3        
7           59.1084     7.49        6.07        0.0909      -5.7        
8           51.6214     7.97        6.98        0.1         -7.3        


/Users/taewankim/Documents/Code/Personal/iLQR/notebooks/../iLQR.py:326: RuntimeWarning: divide by zero encountered in log10
  print("%-12d%-12.6g%-12.3g%-12.3g%-12.3g%-12.1f" % ( iteration,np.sum(self.c),dcost,expected,g_norm,np.log10(self.lamda)) )


9           43.6559     7.13        5.78        0.0906      -inf        
10          36.5222     5.17        4.44        0.0835      -inf        
11          31.3538     4.23        3.77        0.0792      -inf        
12          27.1202     0.0911      3.62        0.0662      -inf        
13          27.0291     5.6         3.48        0.0525      -inf        
14          21.4272     2.71        1.47        0.0382      -inf        
15          18.7177     2.41        1.26        0.0412      -inf        
16          16.3107     2.1         1.04        0.0355      -inf        
17          14.2094     1.9         0.907       0.0294      -inf        
18          12.311      1.56        0.823       0.0274      -inf        
19          10.7514     0.954       0.594       0.0231      -inf        
20          9.79726     0.43        0.287       0.0145      -inf        
21          9.36749     0.149       0.101       0.00809     -inf        
22          9.21813     0.0444      0.0297      0.0

In [ ]:
plt.figure(figsize=(10,10))
fS = 18
plt.subplot(221)
plt.plot(x[:,0], x[:,1], linewidth=2.0)
plt.plot(x_t[0],x_t[1],"o",label='goal')
plt.gca().set_aspect('equal', adjustable='box')
plt.axis([-2, 2, 0, 4.0])
plt.xlabel('X (m)', fontsize = fS)
plt.ylabel('Y (m)', fontsize = fS)
plt.subplot(222)
plt.plot(np.array(range(N+1))*0.1, x[:,0], linewidth=2.0,label='naive')
plt.xlabel('time (s)', fontsize = fS)
plt.ylabel('x1 (m)', fontsize = fS)
plt.subplot(223)
plt.plot(np.array(range(N+1))*0.1, x[:,1], linewidth=2.0,label='naive')
plt.xlabel('time (s)', fontsize = fS)
plt.ylabel('x2 (m)', fontsize = fS)
plt.subplot(224)
plt.plot(np.array(range(N+1))*0.1, x[:,2], linewidth=2.0,label='naive')
plt.xlabel('time (s)', fontsize = fS)
plt.ylabel('x3 (rad)', fontsize = fS)
plt.legend(fontsize=fS)
plt.show()

plt.figure()
plt.subplot(121)
plt.plot(np.array(range(N))*0.1, u[:,0], linewidth=2.0)
plt.xlabel('time (s)', fontsize = fS)
plt.ylabel('v (m/s)', fontsize = fS)
plt.subplot(122)
plt.plot(np.array(range(N))*0.1, u[:,1], linewidth=2.0)
plt.xlabel('time (s)', fontsize = fS)
plt.ylabel('w (rad/s)', fontsize = fS)
plt.show()



# Test integration

In [ ]:
x_test = np.zeros_like(x)

In [ ]:
# using for loop
x_test[0,:] = x[0,:]
state_init = x[0,:]
start = time.time()
for i in range(N) :
    state_init = x_test[i,:]
    int_result = solve_ivp(lambda t,x: myModel.forwardDyn(x,u[i,:],idx=t,discrete=False),
                          t_span=(0,0.1),y0=state_init,max_step=1e-2)
    x_test[i+1,:] = int_result.y[:,-1]
print(time.time()-start)

In [ ]:

# parallel integration
def input_by_time(u,t,t_final) :
    if t >= t_final :
        return u[-1,:]
    else :
        return u[int(t*10),:]
t_eval = np.array(range(N+1))*0.1
state_init = x[0,:]
start = time.time()
int_result = solve_ivp(lambda t,x: myModel.forwardDyn(x,input_by_time(u,t,50),idx=t,discrete=False),
                      t_span=(t_eval[0],t_eval[-1]),
                       t_eval = t_eval,
                       y0=state_init,
                       max_step=1e-2,
                      )
x_test_p = int_result.y.transpose()
print(time.time()-start)

In [ ]:
%matplotlib qt
plt.figure(figsize=(15,5))
plt.subplot(131)
plt.plot(np.array(range(N+1))*0.1, x[:,0], linewidth=2.0,label='naive')
plt.plot(np.array(range(N+1))*0.1, x_test[:,0], linewidth=2.0,label='ode')
plt.plot(np.array(range(N+1))*0.1, x_test_p[:,0], linewidth=2.0,label='ode_parallel')
plt.xlabel('time (s)', fontsize = fS)
plt.ylabel('x1 (m)', fontsize = fS)
plt.subplot(132)
plt.plot(np.array(range(N+1))*0.1, x[:,1], linewidth=2.0,label='naive')
plt.plot(np.array(range(N+1))*0.1, x_test[:,1], linewidth=2.0,label='ode')
plt.plot(np.array(range(N+1))*0.1, x_test_p[:,1], linewidth=2.0,label='ode_parallel')
plt.xlabel('time (s)', fontsize = fS)
plt.ylabel('x2 (m)', fontsize = fS)
plt.subplot(133)
plt.plot(np.array(range(N+1))*0.1, x[:,2], linewidth=2.0,label='naive')
plt.plot(np.array(range(N+1))*0.1, x_test[:,2], linewidth=2.0,label='ode')
# plt.plot(np.array(range(N+1))*0.1, x_test_p[:,2], linewidth=2.0,label='ode_parallel')
plt.xlabel('time (s)', fontsize = fS)
plt.ylabel('x3 (rad)', fontsize = fS)
plt.legend(fontsize=fS)
plt.show()